<a href="https://colab.research.google.com/github/sergekamanzi/Chat-Bot-/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# For encoding questions using sentence embeddings
from sentence_transformers import SentenceTransformer
# For handling vectors and arrays
import numpy as np
# For indexing and searching through the vectors
import faiss
# Optional: for an interactive UI in Colab (Gradio)
import gradio as gr

In [8]:
data = {
    "What was the significance of 1957 in Rwanda?": "In 1957, the Hutu Manifesto was published by Hutu intellectuals demanding political reform and majority rule, marking a turning point in ethnic tensions.",
    "What happened during the Rwandan Revolution of 1959?": "The Rwandan Revolution in 1959 led to the end of the Tutsi monarchy and the rise of a Hutu-led republic. It triggered ethnic violence and the exile of many Tutsis.",
    "When did Rwanda gain independence?": "Rwanda gained independence from Belgium on July 1, 1962, with Grégoire Kayibanda becoming the first president.",
    "Who was Grégoire Kayibanda?": "Grégoire Kayibanda was Rwanda's first president from 1962 to 1973. He led the country after independence and promoted Hutu nationalism.",
    "What happened in 1973 in Rwanda?": "In 1973, Major General Juvénal Habyarimana overthrew Grégoire Kayibanda in a military coup and established a one-party state under the MRND party.",
    "Who was Juvénal Habyarimana?": "Juvénal Habyarimana was Rwanda's second president, ruling from 1973 until his assassination in 1994. His regime was marked by ethnic favoritism and authoritarianism.",
    "What caused the 1994 Genocide?": "The 1994 Genocide against the Tutsi was caused by long-standing ethnic tensions, political instability, and extremist propaganda. The assassination of President Habyarimana triggered the mass killings.",
    "What happened during the 1994 Genocide in Rwanda?": "From April to July 1994, approximately 800,000 Tutsi and moderate Hutu were killed in a genocide perpetrated by Hutu extremists.",
    "Who ended the 1994 Genocide in Rwanda?": "The Rwandan Patriotic Front (RPF), led by Paul Kagame, ended the genocide in July 1994 after capturing Kigali and taking control of the country.",
    "What is the Rwandan Patriotic Front (RPF)?": "The RPF is a political and military organization that ended the 1994 Genocide and has led the Rwandan government since then. It was founded by Tutsi exiles in Uganda.",
    "Who is Paul Kagame?": "Paul Kagame is Rwanda's current president and former military leader of the RPF. He played a key role in ending the 1994 Genocide and has been president since 2000.",
    "What happened in Rwanda after the Genocide?": "After the Genocide, Rwanda focused on national unity, justice through Gacaca courts, reconstruction, and economic development under RPF leadership.",
    "What are Gacaca courts in Rwanda?": "Gacaca courts were community-based courts used to try suspects of the 1994 Genocide, aimed at truth-telling, justice, and reconciliation.",
    "When did Paul Kagame become president?": "Paul Kagame officially became president of Rwanda in 2000 after Pasteur Bizimungu resigned.",
    "What has Rwanda achieved since 2000?": "Since 2000, Rwanda has made progress in healthcare, education, digital transformation, gender equality, and economic development.",
    "What is Vision 2020 in Rwanda?": "Vision 2020 was a national development plan launched by the Rwandan government to transform the country into a middle-income economy by the year 2020.",
    "What replaced Vision 2020 in Rwanda?": "Vision 2050 replaced Vision 2020 and aims to make Rwanda an upper-middle-income country by 2035 and a high-income country by 2050.",
    "What is Umuganda in Rwanda?": "Umuganda is a monthly national community service day in Rwanda where citizens clean streets, plant trees, and build public infrastructure.",
    "What is Rwanda known for today?": "Rwanda is known for its clean cities, rapid development, eco-tourism, commitment to reconciliation, and being one of the safest countries in Africa.",
    "What is Kwibuka in Rwanda?": "Kwibuka is an annual commemoration of the 1994 Genocide against the Tutsi. It means 'to remember' in Kinyarwanda and begins every April 7."
}
